# Segmenting and Clustering Neighborhoods

### Import Libraries 

In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis

### Load and explore the Toronto data

In [15]:
# Use read_html which return is a list of DataFrame objects, even if there is only a single table contained in the HTML content 
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
table=pd.read_html(url)
#convert into a dataframe
df1= pd.DataFrame(table[0]) #need to output the first list as read_html grabs more than one table and we are interested in the first table of the page

In [16]:
df1 = df1[df1.Borough !='Not assigned']
#To reset the index after dropping the rows
df1 = df.reset_index(drop=True)
#To view the dataframe
df1.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [17]:
#To print the number of rows and columns of dataframe.
df.shape

(103, 3)

### Load the geographical coordinates of each postal code

In [18]:
df2=pd.read_csv('http://cocl.us/Geospatial_data')
df2.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [20]:
df3=pd.merge(df1, df2, left_on="Postal Code", right_on="Postal Code")
df3.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
